In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import json
import pickle
import os

from cytassist_segmentation import get_cytassist_subsets_dict_and_cut_cell_detections
import torch

for file in os.listdir("input_data/all_visium_data/preprocessed"):
    if not (file.startswith("cytassist_") and file.endswith("_40x.pkl")):
        continue
    
    slide_id = file.split("_")[1]
    print("Loading", slide_id)

    # This is the original slide
    with open(f"./input_data/all_visium_data/preprocessed/cytassist_{slide_id}_40x.pkl", "rb") as f:
        slide = pickle.load(f)

    # This stores information on how the patients are divided
    with open(f"./input_data/all_visium_data/annotations/json_files/_SS12251_{slide_id}.json", "rb") as f:
        annotation = json.load(f)
        
    patient_subsets, cut_cell_detections = get_cytassist_subsets_dict_and_cut_cell_detections(
        slide,
        annotation,
        # Cell detections
        torch.load(f'./cell-detections/cytassist_{slide_id}/inferences.pt'),
        # Image size surrounding cell detections
        512
    )
    
    # Now, save the resulting patients.
    for patient_id, patient_subset in patient_subsets.items():
        patient_slide = slide.select_subset(patient_subset)
        cell_det = [cut_cell_detections[i] for i in patient_subset]
        print(patient_id, type(patient_slide), type(cell_det)) # Should be <class 'omicsio.datasets.Slide'>
        with open(f"./input_data/all_visium_data/preprocessed/cytassist_split/{slide_id}_{patient_id}.pkl", "wb") as f:
            pickle.dump(patient_slide, f)
            
        torch.save(cell_det, f"./input_data/all_visium_data/preprocessed/cytassist_split/{slide_id}_{patient_id}_cells.pt")

Loading 092842
17 <class 'omicsio.datasets.Slide'> <class 'list'>
16 <class 'omicsio.datasets.Slide'> <class 'list'>
Loading 092146
33 <class 'omicsio.datasets.Slide'> <class 'list'>
3 <class 'omicsio.datasets.Slide'> <class 'list'>
Loading 092534
24 <class 'omicsio.datasets.Slide'> <class 'list'>
35 <class 'omicsio.datasets.Slide'> <class 'list'>
Loading 091759
4 <class 'omicsio.datasets.Slide'> <class 'list'>
7 <class 'omicsio.datasets.Slide'> <class 'list'>


In [8]:
import importlib
import cytassist_segmentation
importlib.reload(cytassist_segmentation)


<module 'cytassist_segmentation' from '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/spatial_omics/cell-detection/cytassist_segmentation.py'>

In [3]:
slide.select_subset

<bound method Slide.select_subset of <omicsio.datasets.Slide object at 0x2b6033eb3f70>>